# Prepare dataset of main process

In [1]:
path = '/Users/davide/Documents/universita/tesi/src/survival_process'
setwd(path)

require(data.table)
require(lubridate)

source('../preprocess/utils/minor.R')

Loading required package: data.table
Loading required package: lubridate

Attaching package: ‘lubridate’

The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following object is masked from ‘package:base’:

    date



## Data loading

In [2]:
# Load data
load('../../data/dati_HF_sample.RData')
# Rename dataset
df = bla
rm(bla)

# load preprocessed dataset to select patients
load('../../data/preprocessed_data.RData')
patients = unique(sel_df$id)

# load fpca scores
load('../../data/score_fpca.RData')

## Preprocessing

In [3]:
# select rows relative to selected patients
df = df[COD_REG %in% patients]
# add time_event column
df[,'time_event' := data_studio_out - (data_rif_ev + make_difftime('days' = 366))]
# add age_in (before follow-up)
df = add_age_in(df,verbose = TRUE)
# add one year (time of follow-up):
df[,'age_in' := age_in+1]

[1] "Adding age_in column (age at t=0, i.e. at dismission from first hospitalization)"
  |======================================================================| 100%

In [4]:
# group by patient
new_df = df[,list(
    'sex' = first(SESSO),
    'age_in' = first(age_in),
    'time_event' = first(time_event),
    'status' = first(desc_studio_out)
  ),by=c("COD_REG")]
colnames(new_df)[1] = 'id'

In [5]:
# add fpca scores
setorderv(new_df, c('id')) 
setorderv(score, c('id')) 

new_df[,'PC1' := score$PC1]
new_df[,'PC2' := score$PC2]

In [6]:
# change columns order
columns = c('id','sex','age_in','PC1','PC2','time_event','status')
new_df = new_df[,..columns]

In [16]:
# minors
new_df[,status:=ifelse(status== 'DECEDUTO',1,0)]
new_df[,time_event:=as.numeric(new_df$time_event)]

In [17]:
head(new_df)

id,sex,age_in,PC1,PC2,time_event,status
10000717,M,73,-10.299129,-3.34178201,705,0
10003004,F,75,-8.944040,-0.44160539,1658,1
10006065,M,69,7.011157,3.96387733,1950,0
10007000,F,84,-14.379666,0.06735727,238,1
10007024,M,71,-6.815942,0.15195553,1914,0
10007217,F,75,-13.031433,-0.58113572,1172,0


In [21]:
save(new_df, file = '../../data/main_process_preprocessed_data.RData')
write.csv(new_df, file = '../../data/main_process_preprocessed_data.csv',row.names=FALSE)